In [3]:
#Original Code

import pycxsimulator
from pylab import *
import copy as cp

prey0 = 100 # initial prey population
K = 500. # prey carrying capacity
prey_m = 0.03 # prey max movement size 
prey_d = 1.0 # prey death rate if close to predator
prey_r = 0.1 # reproduction rate of prey

pred0 = 30 # initial predator population
pred_m = 0.05 # predator max movement size
pred_d = 0.1 # predator death rate by starvation 
pred_r = 0.5 # reproduction rate of predator

r = 0.02 # radius of proximity

class agent: # define agent class for preys and predators
    pass

def initialise():
    # we need to start with empty lists of
    # agents, and data on prey and predators
    # global elements so that their values can be used by other functions
    global agents, preydata, preddata
    agents = []
    preydata = []
    preddata = []
    for i in range(prey0 + pred0):
    # prey0 + pred0 is total number of agents    
        ag = agent() 
        ag.type = 'prey' if i < prey0 else 'pred' 
        # creates prey0 preys and pred0 predators
        ag.x = random() # creates them in random x,y positions
        ag.y = random()
        agents.append(ag) # appends each created ag to agents list

def observe():
    global agents, preydata, preddata # the three lists required for plotting
    
    subplot(1, 2, 1) # subplot: (rows=1, columns=2, position=1 or left) 
    cla() # clear previous plot
    preys = [ag for ag in agents if ag.type == 'prey'] # select preys
    if len(preys) > 0: # if there is still any prey alive
        x = [ag.x for ag in preys] # plot in space
        y = [ag.y for ag in preys]
        plot(x, y, 'b.') # plot in blue, small circle
    predators = [ag for ag in agents if ag.type == 'pred'] #select predators
    if len(predators) > 0:
        x = [ag.x for ag in predators]
        y = [ag.y for ag in predators]
        plot(x, y, 'ro') # red, large circles
    axis('image') # fits plot into limits below
    axis([0, 1, 0, 1]) # x (0, 1) and y (0, 1) limits 

    subplot(1, 2, 2) # rows 1, columns 2, position 2 =  right
    cla()
    plot(preydata, label = 'prey') # plots prey and pred populations
    plot(preddata, label = 'predator')
    legend() # show label above as legend

def update_agent(): # this will update one agent asynchronously, not all live agents
    global agents
    if agents == []: # if all agents are dead, do nothing
        return
    ag = choice(agents) # otherwise select an agent to update

    # simulating random movement
    m = prey_m if ag.type == 'prey' else pred_m # select rates
    ag.x += uniform(-m, m)
    ag.y += uniform(-m, m)
    ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x # agent must stay within plot area 
    ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y

    # detecting a close agent
    neighbours = [nb for nb in agents if nb.type != ag.type 
                 and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < r**2]
                # above: if distance to an agent of the other 
                # type is less than radius, include it in neighbour list
    if ag.type == 'prey': # if agent is prey
        if len(neighbours) > 0: # if there are predators nearby
            if random() < prey_d: # agent dies with proability prey_d
                agents.remove(ag)
                return
        if random() < prey_r*(1-sum([1 for x in agents if x.type == 'prey'])/K):
            agents.append(cp.copy(ag))
            # above: prey reproduces with rate prey_r,
            # times how close it is to carrying capacity
            # offspring is a copy of parent, and is appended to agents list
    else: # if agent is predator
        if len(neighbours) == 0: # if there are no preys nearby
            if random() < pred_d: # predator dies with rate pred_d
                agents.remove(ag)
                return
        else: # if there are prey nearby
            if random() < pred_r: # predator reproduces with rate pred_r
                agents.append(cp.copy(ag)) # offspring is a copy of parent and added to agents list 
               
# This will lead to pseudo-synchronicity -> every agent is updated before next observation
def update():
    global agents, preydata, preddata
    t = 0.
    while t < 1. and len(agents) > 0:
        t += 1. / len(agents)
        update_agent()


    preydata.append(sum([1 for x in agents if x.type == 'prey'])) # counts agents per type in each round
    preddata.append(sum([1 for x in agents if x.type == 'pred']))

pycxsimulator.GUI().start(func=[initialise, observe, update]) # use "update" for pseudo-synchonicity, use "update_agent" for asynchronicity

In [3]:
#My Version

import pycxsimulator
from pylab import *
import copy as cp
import statistics as stats

mut_m = 1 #chances for mutations in movement or reproduction to happen
mut_r = 1

mutr_m = 0.05 #max size of mutation
mutr_r = 0.05

prey0 = 100 # initial prey population
K = 500. # prey carrying capacity
prey_m = 0.03 # prey max movement size 
prey_d = 1.0 # prey death rate if close to predator
prey_r = 0.1 # reproduction rate of prey

pred0 = 30 # initial predator population
pred_m = 0.05 # predator max movement size
pred_d = 0.1 # predator death rate by starvation 
pred_r = 0.5 # reproduction rate of predator

r = 0.02 # radius of proximity

class agent: # define agent class for preys and predators
    pass

def initialise():
    # we need to start with empty lists of
    # agents, and data on prey and predators
    # global elements so that their values can be used by other functions
    global agents, preydata, preddata, preymutdata, predmutdata
    agents = []
    preydata = []
    preddata = []
    preymutdata = [[],[]]
    predmutdata = [[],[]]
    for i in range(prey0 + pred0):
    # prey0 + pred0 is total number of agents    
        ag = agent() 
        if i < prey0:
            ag.type = 'prey'
            ag.m = prey_m
            ag.d = prey_d
            ag.r = prey_r
        else:
            ag.type = 'pred'
            ag.m = pred_m
            ag.d = pred_d
            ag.r = pred_r
        # creates prey0 preys and pred0 predators
        ag.x = random() # creates them in random x,y positions
        ag.y = random()
        agents.append(ag) # appends each created ag to agents list

def observe():
    global agents, preydata, preddata # the three lists required for plotting
    
    subplot(2, 2, 1) # subplot: (rows=1, columns=3, position=1 or left) 
    cla() # clear previous plot
    preys = [ag for ag in agents if ag.type == 'prey'] # select preys
    if len(preys) > 0: # if there is still any prey alive
        x = [ag.x for ag in preys] # plot in space
        y = [ag.y for ag in preys]
        plot(x, y, 'b.') # plot in blue, small circle
    predators = [ag for ag in agents if ag.type == 'pred'] #select predators
    if len(predators) > 0:
        x = [ag.x for ag in predators]
        y = [ag.y for ag in predators]
        plot(x, y, 'ro') # red, large circles
    axis('image') # fits plot into limits below
    axis([0, 1, 0, 1]) # x (0, 1) and y (0, 1) limits 

    subplot(2, 2, 2) # rows 1, columns 3, position 2 =  right
    cla()
    plot(preydata, label = 'prey') # plots prey and pred populations
    plot(preddata, label = 'predator')
    legend() # show label above as legend

    subplot(2, 2, 3)
    cla()
    plot(preymutdata[0], label = 'prey m mean') 
    plot(predmutdata[0], label = 'predator m mean')
    legend() # show label above as legend
    
    subplot(2, 2, 4)
    cla()
    plot(preymutdata[1], label = 'prey r mean') 
    plot(predmutdata[1], label = 'predator r mean')
    legend() # show label above as legend

def update_agent(): # this will update one agent asynchronously, not all live agents
    global agents
    if agents == []: # if all agents are dead, do nothing
        return
    ag = choice(agents) # otherwise select an agent to update

    # simulating random movement
    m = prey_m if ag.type == 'prey' else pred_m # select rates
    ag.x += uniform(-m, m)
    ag.y += uniform(-m, m)
    ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x # agent must stay within plot area 
    ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y

    # detecting a close agent
    neighbours = [nb for nb in agents if nb.type != ag.type 
                 and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < r**2]
                # above: if distance to an agent of the other 
                # type is less than radius, include it in neighbour list
    if ag.type == 'prey': # if agent is prey
        if len(neighbours) > 0: # if there are predators nearby
            if random() < prey_d: # agent dies with proability prey_d
                agents.remove(ag)
                return
        if random() < prey_r*(1-sum([1 for x in agents if x.type == 'prey'])/K):
            offspring = cp.copy(ag)
            if random() < mut_m:
                offspring.m = ag.m + uniform(-mutr_m, mutr_m)
            if random() < mut_r:
                offspring.r = ag.r + uniform(-mutr_r, mutr_r)
            agents.append(offspring)
            # above: prey reproduces with rate prey_r,
            # times how close it is to carrying capacity
    else: # if agent is predator
        if len(neighbours) == 0: # if there are no preys nearby
            if random() < pred_d: # predator dies with rate pred_d
                agents.remove(ag)
                return
        else: # if there are prey nearby
            if random() < pred_r: # predator reproduces with rate pred_r
                offspring = cp.copy(ag)
                if random() < mut_m:
                    offspring.m = ag.m + uniform(-mutr_m, mutr_m)
                if random() < mut_r:
                    offspring.r = ag.r + uniform(-mutr_r, mutr_r)
                agents.append(offspring)
               
# This will lead to pseudo-synchronicity -> every agent is updated before next observation
def update():
    global agents, preydata, preddata
    t = 0.
    while t < 1. and len(agents) > 0:
        t += 1. / len(agents)
        update_agent()


    preydata.append(sum([1 for x in agents if x.type == 'prey'])) # counts agents per type in each round
    preddata.append(sum([1 for x in agents if x.type == 'pred']))
    preymutdata[0].append(stats.mean(x.m for x in agents if x.type == 'prey')) #mean m for prey
    preymutdata[1].append(stats.mean(x.r for x in agents if x.type == 'prey')) #mean r for prey
    predmutdata[0].append(stats.mean(x.m for x in agents if x.type == 'pred')) #mean m for pred
    predmutdata[1].append(stats.mean(x.r for x in agents if x.type == 'pred')) #mean r for pred

pycxsimulator.GUI().start(func=[initialise, observe, update]) # use "update" for pseudo-synchonicity, use "update_agent" for asynchronicity

In [ ]:
#His evolutionary version

import pycxsimulator
from pylab import *
import copy as cp

prey0 = 400 # initial prey population
K = 500. # carrying capacity of prey
prey_m = 0.03 # magnitude of movement of prey
prey_d = 0.8 # death rate of prey
prey_r = 0.1 # reproduction rate of prey

pred0 = 100 # initial predator population
MaxPred = 1000 # max number of predators
pred_m = 0.05 # magnitude of movement of predators
pred_d = 0.1 # death rate of predators
pred_r = 0.5 # reproduction rate of predators

r = 0.02 # radius for proximity detection
mut = 0.01 # mutation rate

class agent:
    pass

def initialise():
    # lists for agents, populations, mobility, and reproduction
    global agents, preydata, preddata, preymob, predmob, preyrep, predrep 
    agents = []
    preydata = []
    preddata = []
    preymob = []
    predmob = []
    preyrep = []
    predrep = []
    
    for i in range(prey0 + pred0):
        ag = agent()
        ag.type = 'prey' if i < prey0 else 'pred'
        ag.x = random()
        ag.y = random()
        ag.m = prey_m if i < prey0 else pred_m
        ag.r = prey_r if i < prey0 else pred_r
        agents.append(ag)

def observe():
    global agents, preydata, preddata, preymob, predmob, preyrep, predrep
    
    subplot(2, 2, 1) # now we need 2 rows and 2 columns for 4 plots
    cla()
    preys = [ag for ag in agents if ag.type == 'prey']
    if len(preys) > 0:
        x = [ag.x for ag in preys]
        y = [ag.y for ag in preys]
        plot(x, y, color='silver', marker='o', ls='') 
        # another way of setting colour and style, 
        # ls is lines,which we dont want 
    predators = [ag for ag in agents if ag.type == 'pred']
    if len(predators) > 0:
        x = [ag.x for ag in predators]
        y = [ag.y for ag in predators]
        plot(x, y, 'ro')
    axis('image')
    axis([-0.1, 1.1, -0.1, 1.1])

    subplot(2, 2, 2) # population plot
    cla()
    plot(preydata, label = 'prey')
    plot(preddata, label = 'predator')
    title('Population')
    legend()

    subplot(2, 2, 3) # movement rate plot
    cla()
    plot(preymob, label = 'prey')
    plot(predmob, label = 'predator')
    title('Movement rate')
    legend()
    
    subplot(2, 2, 4) # reproduction rate plot
    cla()
    plot(preyrep, label = 'prey')
    plot(predrep, label = 'predator')
    title('Reproductive rate')
    legend()
    
def update_agent():
    global agents
    if agents == []:
        return
    ag = choice(agents)

   # detecting neighbours
    neighbours = [nb for nb in agents if nb.type != ag.type
                 and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < r**2]

    if ag.type == 'prey':
        mprey = ag.m
        rprey = ag.r
        if len(neighbours) > 0: # if there are predators nearby
            if random() < prey_d:
                agents.remove(ag)
                return
            else: # run away!
                ag.x += uniform(-mprey, mprey)
                ag.y += uniform(-mprey, mprey)
                ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x
                ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y        
        else: # reproduce if there are no predators
            if random() < rprey*(1-sum([1 for x in agents if x.type == 'prey'])/K):
                offspring = cp.copy(ag)
                offspring.m = ag.m + uniform(-mut, mut) # offspring movement mutates
                offspring.r = ag.r + uniform(-mut, mut) # offspring reproductive rate mutates
                agents.append(offspring)
            ag.x += uniform(-mprey, mprey) # agent moves at slower rate if there are no predators
            ag.y += uniform(-mprey, mprey)
            ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x # stay uîside plot
            ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y        
          
    else:
        mpred = ag.m
        rpred = ag.r
        if len(neighbours) == 0: # if there are no prey nearby
            if random() < pred_d:
                agents.remove(ag)
                return
            else: # move if no prey
                ag.x += uniform(-mpred, mpred)
                ag.y += uniform(-mpred, mpred)
                ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x
                ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y
        
        else: # if there is prey nearby, stay and reproduce
            if random() < rpred*(1-sum([1 for x in agents if x.type == 'pred'])/MaxPred):
                offspring = cp.copy(ag)
                offspring.m = ag.m + uniform(-mut, mut)
                offspring.r = ag.r + uniform(-mut, mut)
                agents.append(offspring)

def update():
    global agents, preydata, preddata, preymob, predmob
    t = 0.
    while t < 1. and len(agents) > 0:
        t += 1. / len(agents)
        update_agent()

    preydata.append(sum([1 for x in agents if x.type == 'prey'])) # store information on population, movement, reproduction
    preddata.append(sum([1 for x in agents if x.type == 'pred']))
    preymob.append(mean([t.m for t in agents if t.type == 'prey']))
    predmob.append(mean([t.m for t in agents if t.type == 'pred']))
    preyrep.append(mean([u.r for u in agents if u.type == 'prey']))
    predrep.append(mean([u.r for u in agents if u.type == 'pred']))
    

pycxsimulator.GUI().start(func=[initialise, observe, update])